In [1]:
#ch10_01a_Cifar_Continue.ipynb
#模型的儲存與載入

In [2]:
#step0. CNN,Convolutional Neural Network 簡介
#
#step0.1 CNN可分為2大部分: 1.影像特徵提取 2.完全連結神經網路
#        1.影像特徵提取:卷積層1,池化層1,卷積層2,池化層2
#        2.完全連結神經網路: 平坦層(Flatten),隱藏層,輸出層

In [3]:
#step1. Data Preprocess
#cifar-10 Data Preprocess ==> ch09_01_Cifar_Intro.ipynb

In [4]:
#step1.1 import necessary module
from keras.datasets import cifar10
import numpy as np
np.random.seed(10)

C:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
#step1.2 讀取 cifar-10 資料
(x_img_train,y_label_train),(x_img_test,y_label_test)=cifar10.load_data()

In [6]:
#step1.3 檢視 train/test dataset
print("train data x_img_train:",x_img_train.shape," train data y_label_train:",y_label_train.shape)
print("test data x_img_test:",x_img_test.shape," test data y_label_test:",y_label_test.shape)

train data x_img_train: (50000, 32, 32, 3)  train data y_label_train: (50000, 1)
test data x_img_test: (10000, 32, 32, 3)  test data y_label_test: (10000, 1)


In [7]:
#step1.4 將features (影像資料) 標準化
x_img_train_normalize=x_img_train.astype('float32')/255.0
x_img_test_normalize=x_img_test.astype('float32')/255.0

In [8]:
#step1.5 將labels (影像值 [0],[1],...,[9]) Onehot encoding 編碼
#        以np_utils.to_categorical()
from keras.utils import np_utils

In [9]:
y_label_train_OneHot=np_utils.to_categorical(y_label_train)
y_label_test_OneHot=np_utils.to_categorical(y_label_test)

In [10]:
#==============================================================
#step2. 建立模型
#

In [11]:
#step2.1 import necessary module
from keras.models import Sequential #keras Sequential module, https://keras.io/getting-started/sequential-model-guide/
from keras.layers import Dense, Dropout, Activation, Flatten #keras layers module: Dense,Dropout,Activation,Flatten
from keras.layers import Conv2D, MaxPooling2D,ZeroPadding2D

In [12]:
#step2.2 建立 keras的 Sequential model
model=Sequential()

In [13]:
#step2.3 建立卷積層1與池化層1, 以model.add()
#        a.fiters=32 --> 隨機產生32個濾鏡, b.size=(3,3) --> 每個濾鏡大小是 (3x3)
#        c.padding='same' -->設定此卷積運算不改變大小, d.activation='relu' activation function 為'ReLU'
#step2.3.1 建立卷積層1, Conv2D()
model.add(Conv2D(filters=32,kernel_size=(3,3),
                 input_shape=(32,32,3),
                 activation='relu',
                 padding='same'))

In [14]:
#step2.3.2 加入Dropoup層,避免 Overfitting, Dropout()
model.add(Dropout(rate=0.25))

In [15]:
#step2.3.3 建立池化層1, MaxPooling2D, pool_size=(2,2) 縮減取樣,將32x32的影像縮小為16x16,影像數量仍是32個
model.add(MaxPooling2D(pool_size=(2,2)))

In [16]:
#step2.4 建立卷積層2與池化層2
#step2.4.1 建立第2次卷積層, Conv2D(), filters=64 (64個濾鏡)
model.add(Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'))

In [17]:
#step2.4.2 再加入 Dropout 避免 overfitting
model.add(Dropout(0.25))

In [18]:
#step2.4.3 建立池化層2, pool_size=(2,2),執行第二次縮減取樣,將16x16的影像縮小為8x8,影像數量仍為64個
model.add(MaxPooling2D(pool_size=(2,2)))

In [19]:
#============================================================================================
#step2.5 建立神經網路(平坦層、隱藏層、輸出層)

In [20]:
#step2.5.1 建立平坦層, Flatten, 將step2.4.3 池化層2 (64x8x8) 轉化為1維向量 4096 個 float (4096個神經元)
#          並加入一個「脫線層」Dorpout, 隨機脫線(放棄) 25% 的神經元(rate=0.25), 用以避免 overfitting
model.add(Flatten())
model.add(Dropout(rate=0.25))

In [21]:
#step2.5.2 建立一個1024個神經元的隱藏層,使用 Dense(), activation function為 'ReLU'
#          同樣加入一個「脫線層」Dorpout, 隨機脫線(放棄) 25% 的神經元(rate=0.25), 用以避免 overfitting
model.add(Dense(1024,activation='relu'))
model.add(Dropout(rate=0.25))

In [22]:
#step2.5.3 建立輸出層,共10個神經元,對應到0~9共10個影像類別
#          activateion function 為 softmax()
model.add(Dense(10,activation='softmax'))

In [23]:
#==================================================================================
#step2.6 檢視模型的摘要
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4096)              0         
__________

In [24]:
#===================================================================
#step3. 進行訓練
#step3.1 定義訓練方式, 使用model.compile() method
#        loss function --> 'crossentropy'(交叉墒) | optimizer --> 'adam' | metrics(評估方式) --> accuracy 
#        訓練方法說明: tf_ch03_01_linearRegression.py
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [25]:
#step3.1a 先檢視是否有之前訓練的模型權重(weight)
try:
    model.load_weights("cifarCNNModel.h5")
    print("Loading model weights successfully!")
except:
    print("Loading model weight failed!")

Loading model weight failed!


In [26]:
#step3.2 if loading model weight failed, 我們就要重新訓練一個。
#        這次，我們的 epochs 設定為 5 就好
train_history=model.fit(x_img_train_normalize,y_label_train_OneHot,  #train dataset
                        validation_split=0.2,                        #有20%的 validation data
                        epochs=5,                                   #訓練週期=5(次)
                        batch_size=128,                              #每周期批次量=128
                        verbose=1)                                   #verbose 層級 1 

Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [==============================] - 64s 2ms/step - loss: 1.5033 - acc: 0.4587 - val_loss: 1.2785 - val_acc: 0.5814
Epoch 2/5
40000/40000 [==============================] - 63s 2ms/step - loss: 1.1385 - acc: 0.5954 - val_loss: 1.1070 - val_acc: 0.6424
Epoch 3/5
40000/40000 [==============================] - 63s 2ms/step - loss: 0.9837 - acc: 0.6549 - val_loss: 1.0193 - val_acc: 0.6591
Epoch 4/5
40000/40000 [==============================] - 63s 2ms/step - loss: 0.8748 - acc: 0.6924 - val_loss: 0.9535 - val_acc: 0.6956
Epoch 5/5
40000/40000 [==============================] - 63s 2ms/step - loss: 0.7834 - acc: 0.7260 - val_loss: 0.8851 - val_acc: 0.7033


In [27]:
#step3.3 訓練結束後(5個epochs後),使用 model.save_weights 儲存訓練後的weghts 
model.save_weights("cifarCNNModel.h5") #在現行目錄

In [28]:
#檢視檔案
!dir *.h5

 磁碟區 E 中的磁碟沒有標籤。
 磁碟區序號:  BE0A-1016

 E:\worspace\tensorflow\py 的目錄

2018/09/27  上午 11:35        16,925,000 cifarCNNModel.h5
               1 個檔案      16,925,000 位元組
               0 個目錄  350,160,732,160 位元組可用


In [29]:
#step3.4 重啟訓練，我們先將之前的模型權重檔載入
try:
    model.load_weights("cifarCNNModel.h5")
    print("Loading model weights successfully!")
except:
    print("Loading model weight failed!")

Loading model weights successfully!


In [30]:
#step3.5 模型權重檔載入成功，我們以載入的權重為權重，再作5個epochs訓練
train_history_2=model.fit(x_img_train_normalize,y_label_train_OneHot,  #train dataset
                        validation_split=0.2,                        #有20%的 validation data
                        epochs=5,                                   #訓練週期=5(次)
                        batch_size=128,                              #每周期批次量=128
                        verbose=1)                                   #verbose 層級 1 

Train on 40000 samples, validate on 10000 samples
Epoch 1/5
40000/40000 [==============================] - 63s 2ms/step - loss: 0.7022 - acc: 0.7498 - val_loss: 0.8195 - val_acc: 0.7288
Epoch 2/5
40000/40000 [==============================] - 63s 2ms/step - loss: 0.6191 - acc: 0.7824 - val_loss: 0.8102 - val_acc: 0.7295
Epoch 3/5
40000/40000 [==============================] - 63s 2ms/step - loss: 0.5519 - acc: 0.8041 - val_loss: 0.7823 - val_acc: 0.7423
Epoch 4/5
40000/40000 [==============================] - 63s 2ms/step - loss: 0.4890 - acc: 0.8288 - val_loss: 0.7536 - val_acc: 0.7460
Epoch 5/5
40000/40000 [==============================] - 63s 2ms/step - loss: 0.4237 - acc: 0.8512 - val_loss: 0.7566 - val_acc: 0.7448
